In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer 
import numpy
import nltk, re, string, collections
from nltk.util import ngrams
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score


In [14]:
def read_cleaned_data_set():
    file = open("output/tokens_no_stopwords_train.txt", encoding='utf-8')
    corpus =[]
    lines = file.readlines()
    for line in lines:
        result=line.rstrip('\n')
        if len(result)>0:
            result = result[1:len(result)-2]
            result=result.replace(",","")
            result= result.replace("'","")
        
            corpus.append(result)
    return corpus

In [15]:
corpus =read_cleaned_data_set()

In [16]:
def preprocessing_tf_idf(i,tfIdfVectorizer,tfIdf):
    df = pd.DataFrame(tfIdf[i].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
    #sorting them
    # df = df.sort_values('TF-IDF', ascending=False)
    dict_res = df.to_dict()
    dict_res= dict_res['TF-IDF']
    return dict_res

In [17]:
def get_TF_IDF(docs): 
    #create tf-idf vector & enable idf to calculate them in one steps
    tfIdfVectorizer=TfidfVectorizer(use_idf=True)
    #get values for docs
    tfIdf = tfIdfVectorizer.fit_transform(docs)
   
    # get first tf-idfs for first doc only to be visualized
    # get features names mean get words
    # print(tfIdf)
    df = pd.DataFrame(tfIdf[15].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
    #sorting them
    df = df.sort_values('TF-IDF', ascending=False)
    dict_res = df.to_dict()
    dict_res= dict_res['TF-IDF']
    tf_idf_docs =[]
    for i in range(0,len(docs)):
         dict_res= preprocessing_tf_idf(i,tfIdfVectorizer,tfIdf)
         tf_idf_docs.append(dict_res.values())
   
    # print(df.head(15))
    return dict_res,tfIdf,tf_idf_docs,tfIdfVectorizer

    #show first 15 of them 

    


In [18]:
# dict_res,tfIdf,tf_idf_docs=get_TF_IDF(corpus)
# for i in range(7,tfIdf.shape[0]):
#     if len(tfIdf[i].T.todense())!=15:
#         print(tfIdf[i])
#         break

# print(dict_res)
# print(tf_idf.vocabulary_)

In [19]:
# pd.DataFrame(tf_idf_docs).to_csv("output.csv",index=False)

In [20]:
def assign_tf_idf(corpus,dict_res):
    td_idf_sentences=[]
    for text in corpus:
        tf_idf=[]
        for value in dict_res:
            if value !='قوة':
                if value in text:
                    tf_idf.append(dict_res[value])
                else:
                    tf_idf.append(0.0)
    
        td_idf_sentences.append(tf_idf)
    return td_idf_sentences

In [21]:
# td_idf_sentences=assign_tf_idf(corpus,dict_res)
# print(td_idf_sentences[0])
#changed 

In [22]:
def get_tokens_freq(corpus):
    # corpus_new = corpus.split()
    tokens ={}
    for text in corpus:
        text = text.split()
        for value in text:
            if value in tokens :
                tokens[value]+=1
            else:
                tokens[value]=1
    return tokens

In [23]:
def get_n_grams(corpus, n_gram):
    all_n_grams=[]
    for text in corpus:
        padding = ["باد"]*(n_gram-1)
        text = text.split()
        text = padding+ text+padding
        n_grams = ngrams(text, n_gram)
        all_n_grams+=n_grams
    ngramFreq = collections.Counter(all_n_grams)
    return ngramFreq


In [24]:
def convert_ngrams_probablities( n,corpus):
  probablities =[]
  ngrams= get_n_grams(corpus,n)
  
  # print("kdkdk",ngrams)
  if n>1:
    n_minus_one_grams = get_n_grams(corpus, n-1)
  tokens =get_tokens_freq(corpus)
  for text in corpus:
    padding = ["باد"]*(n-1)
    text_value=text.split()
    text_value= padding+text_value+padding
    prob_list= 1.0
    for index in range(n, len(text_value)-n) :
      n_gram_text = tuple(text_value[index-n:index ])
      n_minus_one_grams_text = tuple(text_value[index-n:index-1])
      #here to add one smooth padding
      nomiantor=1
      dominator=len(tokens) 
      # nomiantor=0.0
      # dominator=0.0
      if n_gram_text in ngrams:
  
        nomiantor +=ngrams[n_gram_text]
      if n_minus_one_grams_text in n_minus_one_grams:
  
        dominator+= n_minus_one_grams[n_minus_one_grams_text]

      elif n==1:
        dominator += tokens[n_minus_one_grams_text]
      prob=nomiantor /dominator
      prob_list*=prob # for one doc
    
    probablities.append(prob_list) # for all docs
    
  return probablities
    

In [25]:
prob=convert_ngrams_probablities(3,corpus)
result=get_n_grams(corpus,2)
# print(result)
# # result.
result.most_common(10)


[(('لقاح', 'كورونا'), 1369),
 (('الجرعة', 'الاولى'), 665),
 (('الاولى', 'لقاح'), 574),
 (('كورونا', 'باد'), 469),
 (('يتلقى', 'الجرعة'), 424),
 (('جرعة', 'لقاح'), 400),
 (('لقاح', 'باد'), 385),
 (('لقاح', 'فايزر'), 333),
 (('فيروس', 'كورونا'), 313),
 (('الجرعة', 'الثانية'), 308)]

In [26]:
print((prob[0]))

1.6597412693274927e-110


In [27]:
print(prob[100])

4.2552940390803975e-123


In [28]:
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec

In [29]:
def get_word_embeddings(corpus, min_count=1, vector_size=100, window=5, sg = 1):
	data = []
	for i in range(len(corpus)):
		temp = []
		for j in corpus[i].split():
			temp.append(j)
		data.append(temp)
	# IMP Note: If the lines below result in an error, change size -> vector_size. It's a version variable naming issue.
	# Create CBOW model
	model1 = gensim.models.Word2Vec(data, min_count=min_count, vector_size =vector_size, window=window)
	# Create Skip Gram model
	model2 = gensim.models.Word2Vec(data, min_count=min_count, vector_size =vector_size, window=window, sg=sg)
	return model1, model2

In [30]:
cbowModel, sgModel = get_word_embeddings(corpus)

In [31]:
# For CBOW
print("Cosine similarity between 'لقاح' " + "and 'كورونا' - CBOW : ", cbowModel.wv.similarity('لقاح', 'كورونا'))
print("Cosine similarity between 'لقاح' " + "and 'صيفي' - CBOW : ", cbowModel.wv.similarity('لقاح', 'صيفي'))
# For Skip Gram
print("Cosine similarity between 'لقاح' " + "and 'كورونا' - Skip Gram : ",	sgModel.wv.similarity('لقاح', 'كورونا'))
print("Cosine similarity between 'لقاح' " + "and 'صيفي' - Skip Gram : ", sgModel.wv.similarity('لقاح', 'صيفي'))

Cosine similarity between 'لقاح' and 'كورونا' - CBOW :  0.9988935
Cosine similarity between 'لقاح' and 'صيفي' - CBOW :  0.77869225
Cosine similarity between 'لقاح' and 'كورونا' - Skip Gram :  0.8618764
Cosine similarity between 'لقاح' and 'صيفي' - Skip Gram :  0.6856279
